# Practical 5

This notebook contains some code to help in the tasks of Project B.

In [2]:
%matplotlib inline

import numpy
from numpy import newaxis as nax  # to simplify the many uses of this

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib_inline.backend_inline import set_matplotlib_formats
from mpl_toolkits.mplot3d import Axes3D

from astropy import units
from astropy import constants
from astropy.table import QTable  # To use tables with units
from astropy.modeling.models import BlackBody
from astropy.visualization import quantity_support

from tqdm import tqdm

quantity_support()
set_matplotlib_formats('svg')
plt.rc('legend', frameon=False)
plt.rc('figure', figsize=(7, 7 / 1.75)) # Larger figure sizes
plt.rc('font', size=12)

from scipy.integrate import cumtrapz
from scipy.ndimage import shift  # for "rotating" 3D cubes
from scipy.special import wofz   # for Voigt function
from scipy.special import gamma  # for ABO broadening

i_units = units.Quantity(1, unit="kW m-2 sr-1 nm-1")  # More practical SI units

We start with a slightly modified version of the `Atom()` class from Project A:

In [3]:
class Atom:
    """
    Reads atomic data, calculates level populations according to Boltzmann's law,
    and ionisation fractions according to Saha's law.
    """
    
    def __init__(self, atomfile=None):
        """
        Parameters
        ----------
        atomfile : string, optional
            Name of file with atomic data. If not present, atomic data needs
            to be loaded with the .read_atom method.
        """
        self.loaded = False
        if atomfile:
            self.read_atom(atomfile)
        
    def read_atom(self, filename):
        """
        Reads atom structure from text file.
        
        Parameters
        ----------
        filename: string
            Name of file with atomic data.
        """
        tmp = numpy.loadtxt(filename, unpack=True)
        self.n_stages = int(tmp[2].max()) + 1
        # Get maximum number of levels in any stage
        self.max_levels = 0
        for i in range(self.n_stages):
            self.max_levels = max(self.max_levels, (tmp[2] == i).sum())
        # Populate level energies and statistical weights
        # Use a square array filled with NaNs for non-existing levels
        chi = numpy.empty((self.n_stages, self.max_levels))
        chi.fill(numpy.nan)
        self.g = numpy.copy(chi)
        for i in range(self.n_stages):
            nlevels = (tmp[2] == i).sum()
            chi[i, :nlevels] = tmp[0][tmp[2] == i]
            self.g[i, :nlevels] = tmp[1][tmp[2] == i]
        # Put units, convert from cm-1 to Joule
        chi = (chi / units.cm).to('aJ', equivalencies=units.spectral())
        # Save ionisation energies, saved as energy of first level in each stage
        self.chi_ion = chi[:, 0].copy()
        # Save level energies relative to ground level in each stage
        self.chi = chi - self.chi_ion[:, nax]
        self.loaded = True

    def compute_partition_function(self, temperature):
        """
        Computes partition functions using the atomic level energies and
        statistical weights.
        
        Parameters
        ----------
        temperature: astropy.units.quantity (scalar or array)
            Gas temperature in units of K or equivalent.
            
        Returns
        -------
        pfunc : astropy.units.quantity (array)
            Partition function. Shape is (nstages,) + temperature.shape.
        """
        if not self.loaded:
            raise ValueError("Missing atom structure, please load atom with read_atom()")
        temp = temperature.T[..., nax, nax]  # to allow broadcast
        return numpy.nansum(self.g.T * numpy.exp(-self.chi.T / constants.k_B / temp), axis=-2).T
    
    def compute_excitation(self, temperature):
        """
        Computes the level populations relative to the ground state,
        according to the Boltzmann law.
        
        Parameters
        ----------
        temperature: astropy.units.quantity (scalar or array)
            Gas temperature in units of K or equivalent.
        
        Returns
        -------
        exc : astropy.units.quantity (array)
            Populations of excited states relative to total number of species.
            Shape is (nstages, nlevels) + temperature.shape.
        """
        pfunc = self.compute_partition_function(temperature)
        temp = temperature.T[..., nax, nax]
        # relative to total number of atoms in this stage
        g_ratio = self.g.T / pfunc[:, nax].T
        result = (g_ratio * numpy.exp(-self.chi.T / (constants.k_B * temp))).T
        return result
    
    def compute_ionisation(self, temperature, electron_density):
        """
        Computes ionisation fractions according to the Saha law.
        IMPORTANT: changed from SSA: now takes electron density 
                   instead of electron pressure.
        
        Parameters
        ----------
        temperature: astropy.units.quantity (scalar or array)
            Gas temperature in units of K or equivalent.
        electron_density: astropy.units.quantity (scalar)
            Electron density in units of m^-3 or equivalent.

        Returns
        -------
        ion_frac : astropy.units.quantity (array)
            Ionisation fraction relative to total number of species.
            Shape is (nstages,) + temperature.shape.
        """
        partition_function = self.compute_partition_function(temperature)
        saha_const = ((2 * numpy.pi * constants.m_e * constants.k_B * 
                       temperature.astype('d')) / (constants.h ** 2)) ** (3 / 2)
        nstage = numpy.zeros_like(partition_function) / units.m ** 3
        nstage[0] += 1. / units.m ** 3
        # Below we use the values for ionisation energies that are saved
        # in the first index of each excited state: self.chi[r + 1, 0]
        for r in range(self.n_stages - 1):
            nstage[r + 1] = (nstage[r] / electron_density * 2 * saha_const *
                             partition_function[r + 1] / partition_function[r] * 
                             numpy.exp(-self.chi_ion[r + 1, nax] / 
                                       (constants.k_B * temperature[nax])))
        # nansum is needed because the last stage might have only one level
        # (only ionisation potential)
        return nstage / numpy.nansum(nstage, axis=0)

    def compute_populations(self, temperature, electron_density):
        """
        Computes relative level populations for all levels and all
        ionisation stages using the Bolzmann and Saha laws.
        IMPORTANT: project A: now takes electron density 
                   instead of electron pressure.
        
        Parameters
        ----------
        temperature: astropy.units.quantity (scalar or array)
            Gas temperature in units of K or equivalent.
        electron_density: astropy.units.quantity (scalar)
            Electron density in units of m^-3 or equivalent.
            
        Returns
        -------
        pop_frac : astropy.units.quantity (array)
            Population of different levels relative to total number of species.
            Shape is (nstages, nlevels) + temperature.shape.
        """
        return (self.compute_excitation(temperature) * 
                self.compute_ionisation(temperature, electron_density)[:, nax])

And now some functions to compute the line broadening: Doppler, natural, and van der Waals:

In [4]:
def compute_doppler_width(wave0, temperature, mass, vturb):
    """
    Computes the Doppler width.
    
    Parameters
    ----------
    wave0: astropy.units.quantity (scalar)
        Rest wavelength of the bound-bound transition, in units of length.
    temperature: astropy.units.quantity (scalar or array)
        Temperature(s) where to calculate the Doppler width.
    mass: astropy.units.quantity (scalar)
        Mass of the atomic species, in units of mass.
    vturb: astropy.units.quantity (scalar or array)
        Turbulent velocity in units of length per time. If array, should be
        the same shape as temperature.
        
    Returns
    -------
    doppler_width: astropy.units.quantity (scalar or array)
        Doppler width in units of length. Same shape as temperature.
    """
    return wave0 / constants.c * numpy.sqrt(2 * constants.k_B * 
                                            temperature / mass + vturb ** 2)


def compute_gamma_natural(wave, g_ratio, f_value):
    """
    Computes the natural line damping parameter for a bound-bound transition.
        
    Parameters
    ----------
    wave: astropy.units.quantity (scalar or array)
        Wavelength to calculate.
    g_ratio: float
        Ratio between statistical weights of lower and upper levels (gl / gu)
    f_value: float
        Transition f value.

    Returns
    -------
    gamma - astropy.units.quantity (scalar or array)
        Damping parameter in units of s^-1.
    """
    return 6.67e13 * g_ratio * f_value / wave.to_value('nm') ** 2 / units.s


def compute_gamma_vdW(temperature, h_neutral, alpha, sigma, mass):
    """
    Computes the van der Waals damping parameter for a bound-bound transition using
    the receipe of Barklem/O'Mara/Anstee. 
    
    Parameters
    ----------
    temperature: astropy.units.quantity (scalar or array)
        Gas temperature in units of K or equivalent.
    h_neutral: astropy.units.quantity (scalar or array)
        Number density of neutral hydrogen atoms, in m^-3 or equivalent.
        Same shape as temperature.
    alpha: scalar
        alpha parameter from Barklem/O'Mara/Anstee formalism.
    sigma: scalar
        sigma parameter from Barklem/O'Mara/Anstee formalism.
    mass: astropy.units.quantity (scalar)
        Mass of perturbing atom, in units of mass.
    """
    Ar_H = 1.007975  # Atomic weight of hydrogen
    mu_m = constants.u / (1 / Ar_H + 1 / (mass / constants.u))
    v_bar = numpy.sqrt(8 * constants.k_B * 1 * units.K / (numpy.pi * mu_m))
    v_ratio = (1e4 * units.m / units.s / v_bar).to('')
    const = (2 * constants.a0**2 * (4 / numpy.pi)**(alpha / 2) * 
             gamma((4 - alpha) / 2) * v_bar * sigma * v_ratio**alpha).to("m^3/s")
    return const * temperature.to_value("K") ** ((1 - alpha)/2) * h_neutral

And to compute the line profile:

In [6]:
def voigt(a, u):
    """
    Computes the Voigt function.
    """
    z = (u + 1j * a)
    return wofz(z).real


def compute_profile(wave, wave0, gamma, doppler_width):
    """
    Computes the line profile for a given broadening and wavelength.
    
    Parameters
    ----------
    wave : astropy.units.quantity (scalar)
        Wavelength where to evaluate line profile. In nm or similar.
    wave : astropy.units.quantity (scalar)
        Transition wavelength. In nm or similar.
    gamma: astropy.units.quantity (scalar array)
        Line broadening (gamma of Lorenztian), in units of s^-1. 
        Can be an array of any size (works in 1D and 3D).
    doppler_width: astropy.units.quantity (scalar or array)
        Doppler width in units of wavelength. Must be same shape
        as gamma.
    """
    damping = (wave**2 / (4 * numpy.pi * constants.c * doppler_width) * 
               gamma)
    u = ((wave - wave0) / doppler_width).to("")
    return voigt(damping, u)

Finally, a function to compute the line extinction (but without the line profile). We don't include the line profile for computational efficiency - it is the only part of $\alpha_\lambda^l$ that depends on wavelength, so it makes sense to include it only in a loop.

In [7]:
def compute_NaID_extinction(wave, doppler_width, temperature, electron_density, h_density): 
    """
    Extinction without the profile
    """
    # Load Na model atom and define some additional properties
    na_atom = Atom("NaI_atom.txt")
    abund_Na = 1.7378e-6  # From Asplund et al. (2009)
    mass_Na = 22.99 * constants.u
    f_value = 0.318
    # Lower level populations, neutral stage, ground level    
    pops = na_atom.compute_populations(temperature, electron_density)[0,0]
    pops = pops * abund_Na * h_density
    # Constants in the formula for alpha
    alpha_const = constants.e.si ** 2 / (4 * constants.eps0 * constants.m_e * constants.c**2)
    # Stimulated emission
    stim = (1 - numpy.exp(-constants.h * constants.c / 
                          (wave * constants.k_B * temperature)))
    # Line extinction
    return alpha_const * wave**2 * pops * f_value / doppler_width * stim

These three functions will be worked on in the exercise classes:

In [8]:
def lte_intensity(wavelength, distance, temperature, extinction):
    """
    Solves the radiative transfer equation assuming LTE for a single ray.
    
    Parameters
    ----------
    wavelength: astropy.units.quantity (scalar)
        Wavelength to calculate, in units of length.
    distance : astropy.units.quantity (1-D array)
        Distances along path of ray, in units of length. Can be different
        length than wavelength array.
    temperature: astropy.units.quantity (n-D array)
        Gas temperature in units of K or equivalent, for all points along
        the ray. Same length as distance.
    extinction: astropy.units.quantity (n-D array)
        Extinction coefficient in units of inverse length, for all
        points along the ray. Same dimensions as temperature.
    """
    # your code here

In [ ]:
def compute_NaID_intensity(temperature, electron_density, h_density, vturb, height):
    """
    Calculates the emergent intensity for the Na I D1 line for a given 1D atmosphere.
    
    Parameters
    ----------
    temperature: astropy.units.quantity (1-D array)
        Gas temperature in units of K or equivalent.
    electron_density: astropy.units.quantity (1-D array)
        Electron density in units of m^-3 or equivalent.    
    h_density: astropy.units.quantity (1-D array)
        Total hydrogen number density in units of m^-3 or equivalent.   
    vturb: astropy.units.quantity (1-D array)
        Turbulent velocity in units of m/s or equivalent.
    height: astropy.units.quantity (1-D array)
        Height scale of the model atmosphere in units of m or equivalent.
    """
    # your code here

In [ ]:
def shift_simulation(data, mu):
    """
    Shifts (or translates) a 3D horizontally periodic simulation according to an
    inclination angle given by mu = cos(angle). 
    
    Parameters
    ----------
    data : astropy.table.table.QTable
        Data from a 3D simulation in QTable format. Should have 
        several 3D variables plus a single 1D variable called "height",
        and some extra metadata.
    mu : float
        Cosine of inclination angle.
    """
    # your code here